In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import math


mnist = input_data.read_data_sets("/home/zhaocq/桌面/tensorflow/mnist/raw/",one_hot=True)
sess = tf.InteractiveSession()

batchs = 64
df_dim = 64#鉴别器通道
gf_dim = 64#生成器通道
z_dim = 100
output_height = 28
output_weight = 28
image_dims = [output_height,output_weight,1]
inputs = tf.placeholder(tf.float32,[batchs]+image_dims,name = "real_images")
z = tf.placeholder(tf.float32,[None,z_dim],name = 'z')
x = tf.placeholder(tf.float32,[batchs,784],name = 'x')
x_image = tf.reshape(x ,[batchs,28,28,1])


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/t10k-images-idx3-ubyte.gz
Extracting /home/zhaocq/桌面/tensorflow/mnist/raw/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:


def conv2d(input_,output_dim,k_h=5,k_w=5,d_h=2,d_w=2,steddev=0.02,name="conv2d"):
    with tf.variable_scope(name):
        w = tf.get_variable('w',[k_h,k_w,input_.get_shape()[-1],output_dim],\
                            initializer= tf.truncated_normal_initializer(stddev=steddev))
        conv = tf.nn.conv2d(input_,w,strides=[1,d_h,d_w,1],padding='SAME')
        biases = tf.get_variable('biases',[output_dim],initializer=tf.constant_initializer(0.0))
        conv = tf.reshape(tf.nn.bias_add(conv,biases),conv.get_shape())
        return conv

def deconv2d(input_,output_shape,k_h=5,k_w=5,d_h=2,d_w=2,steddev=0.02,name="deconv2d"):
    with tf.variable_scope(name):
        w = tf.get_variable('w',[k_h,k_w,output_shape[-1],input_.get_shape()[-1]],\
                           initializer=tf.random_normal_initializer(stddev=steddev))
        deconv = tf.nn.conv2d_transpose(input_,w,output_shape=output_shape,strides=[1,d_h,d_w,1])
        biases = tf.get_variable('biases',[output_shape[-1]],initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv,biases),deconv.get_shape())
        return deconv

def linear(input_,output_size,scope=None,steddev=0.02):
    shape = input_.get_shape().as_list()
    with tf.variable_scope(scope or "Linear"):
        matrix = tf.get_variable("matrix",[shape[1],output_size],tf.float32,tf.random_normal_initializer(stddev=steddev))
        bias = tf.get_variable("bias",[output_size],initializer=tf.constant_initializer(0.0))
        return tf.matmul(input_,matrix) + bias

def conv_out_size_same(size,stride):
    return int(math.ceil(float(size) / float(stride)))

def discriminator(image,reuse = False):
    with tf.variable_scope("discriminator") as scope:
        if reuse:
            scope.reuse_variables()
        h0 = tf.nn.leaky_relu(conv2d(image,df_dim,name='d_h0_conv'))
        h1 = tf.nn.leaky_relu(tf.contrib.layers.batch_norm(conv2d(h0,df_dim*2,name='d_h1_conv')))
        h2 = tf.nn.leaky_relu(tf.contrib.layers.batch_norm(conv2d(h1,df_dim*4,name='d_h2_conv')))
        h3 = tf.nn.leaky_relu(tf.contrib.layers.batch_norm(conv2d(h2,df_dim*8,name='d_h3_conv')))
        h4 = linear(tf.reshape(h3,[batchs,-1]),1,'d_h4_lin')
        return tf.nn.sigmoid(h4),h4
def generator(z):
    with tf.variable_scope("generator") as scope:
        s_h,s_w = output_height,output_weight
        s_h2,s_w2 = conv_out_size_same(s_h,2),conv_out_size_same(s_w,2)
        s_h4,s_w4 = conv_out_size_same(s_h2,2),conv_out_size_same(s_w2,2)
        s_h8,s_w8 = conv_out_size_same(s_h4,2),conv_out_size_same(s_w4,2)
        s_h16,s_w16 = conv_out_size_same(s_h8,2),conv_out_size_same(s_w8,2)
        
        z_ = linear(z,gf_dim*8*s_h16*s_w16,'g_h0_lin')
        h0 = tf.reshape(z_,[-1,s_h16,s_w16,gf_dim*8])
        h0 = tf.nn.relu(tf.contrib.layers.batch_norm(h0))
        h1 = deconv2d(h0,[batchs,s_h8,s_w8,gf_dim*4],name='g_h1')
        h1 = tf.nn.relu(tf.contrib.layers.batch_norm(h1))
        h2 = deconv2d(h1,[batchs,s_h4,s_w4,gf_dim*2],name='g_h2')
        h2 = tf.nn.relu(tf.contrib.layers.batch_norm(h2))
        h3 = deconv2d(h2,[batchs,s_h2,s_w2,gf_dim*1],name='g_h3')
        h3 = tf.nn.relu(tf.contrib.layers.batch_norm(h3))
        h4 = deconv2d(h3,[batchs,s_h,s_w,1],name='g_h4')
        return tf.nn.sigmoid(h4)

    
g_sample = generator(z)
d_real,d_log_real = discriminator(x_image,reuse=False)
d_fake,d_log_fake = discriminator(g_sample,reuse=True)

t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'd_' in var.name]
g_vars = [var for var in t_vars if 'g_' in var.name]
d_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_log_real,labels=tf.ones_like(d_real)) + \
                        tf.nn.sigmoid_cross_entropy_with_logits(logits=d_log_fake,labels=tf.zeros_like(d_fake)))#交叉熵损失
g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_log_fake,labels=tf.ones_like(d_fake)))
#定义原始损失
#d_loss = - tf.reduce_mean(tf.log(d_real) + tf.log(1.- d_fake))
#g_loss =  - tf.reduce_mean(tf.log(d_fake))
#g_loss =  tf.reduce_mean(tf.square(1.-d_fake)-tf.log(d_fake))
d_slover = tf.train.AdamOptimizer(0.0002,beta1=0.5).minimize(d_loss,var_list = d_vars)
g_slover = tf.train.AdamOptimizer().minimize(g_loss,var_list = g_vars)

In [3]:
#训练过程
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


sess.run(tf.global_variables_initializer())
i=0
for epoch in range (12):
    for it in range(800):
    #输出image
        sample_z = np.random.uniform(-1, 1, [batchs, z_dim]).astype(np.float32)
        x_md,_ =mnist.train.next_batch(batchs)
        _,g_loss_curr = sess.run([g_slover,g_loss],feed_dict={z: sample_z})
        #d_loss_curr = sess.run(d_loss,feed_dict={x: x_md, z: sample_z})
        _,d_loss_curr = sess.run([d_slover,d_loss],feed_dict={x: x_md, z: sample_z})
        _,g_loss_curr = sess.run([g_slover,g_loss],feed_dict={z: sample_z})
        _,g_loss_curr = sess.run([g_slover,g_loss],feed_dict={z: sample_z})
        _,g_loss_curr = sess.run([g_slover,g_loss],feed_dict={z: sample_z})
    if epoch % 1 == 0:
        #print('iter:{}'.format(epoch),'g loss : {:.4}'.format(g_loss_curr))
        sampl = sess.run(g_sample,feed_dict={z: sample_z})
        samples = sampl[:16]
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(5)),bbox_inches = 'tight')
        i+=1
        plt.close(fig)
        print('iter:{}'.format(epoch),'d loss : {:.4}'.format(d_loss_curr),'g loss : {:.4}'.format(g_loss_curr))

iter:0 d loss : 0.1131 g loss : 4.98
iter:1 d loss : 0.02829 g loss : 7.473
iter:2 d loss : 0.439 g loss : 5.176
iter:3 d loss : 0.2809 g loss : 4.791
iter:4 d loss : 0.255 g loss : 2.601
iter:5 d loss : 0.8293 g loss : 0.2511
iter:6 d loss : 0.2299 g loss : 5.164
iter:7 d loss : 0.09369 g loss : 3.981
iter:8 d loss : 0.09259 g loss : 8.178
iter:9 d loss : 0.3603 g loss : 2.516
iter:10 d loss : 0.09593 g loss : 3.587
iter:11 d loss : 0.02535 g loss : 5.384
